This notebook runs a `Qallse` model in simulation mode and shows how to simply visualize the results using utilities defined in module `qallse.plotting`.

# Import and setup

In [1]:
# import modules
import pandas as pd
import numpy as np

from qallse.plotting import *
from qallse import *
from qallse.dsmaker import create_dataset

# initialise the plotting module in "notebook" mode
set_notebook_mode()

# initialise the logging module
import logging
logging.basicConfig()
fmt = logging.Formatter("%(asctime)s.%(msecs)03d %(levelname)s %(name)s: %(message)s", datefmt='%H:%M:%S')
for handler in logging.getLogger().handlers: handler.setFormatter(fmt)
logging.getLogger('hepqpr').setLevel(logging.DEBUG)

# Define run config

In [2]:
# == DATASET CONFIG

dsmaker_config = dict(
    density=.01, # 1% 
)

# == INPUT CONFIG

# whether or not to add missing doublets to the input
add_missing = True 

# == RUN CONFIG

model_class = QallseD0 # model class to use
extra_config = dict() # configuration arguments overriding the defaults

# Generate the data

In [3]:
import tempfile

tempdir = tempfile.TemporaryDirectory()
print(f'using {tempdir.name}')
metas, path = create_dataset(output_path=tempdir.name, gen_doublets=True, **dsmaker_config)

using /var/folders/qp/grrqv98s1c37f8bxtkblfpcr0000gn/T/tmpra3o223v


17:46:22.850 DEBUG qallse.dsmaker.dsmaker: Loaded 120939 hits from /Users/lin/git/quantum-annealing-project/project-2/src/hepqpr/qallse/dsmaker/data/event000001000.
17:46:22.873 DEBUG qallse.dsmaker.dsmaker: Filtered hits from barrel. Remaining hits: 65518.
17:46:22.914 DEBUG qallse.dsmaker.dsmaker: Dropped double hits. Remaining hits: 55446.
17:46:22.935 DEBUG qallse.dsmaker.dsmaker: High Pt hits: 32302/524
17:46:22.967 DEBUG qallse.dsmaker.dsmaker:   num_hits=524
17:46:22.967 DEBUG qallse.dsmaker.dsmaker:   num_tracks=81
17:46:22.968 DEBUG qallse.dsmaker.dsmaker:   num_important_tracks=4
17:46:22.968 DEBUG qallse.dsmaker.dsmaker:   num_noise=130
17:46:22.969 DEBUG qallse.dsmaker.dsmaker:   random_seed=1547862382
17:46:22.969 DEBUG qallse.dsmaker.dsmaker:   time=2019-01-18T17:46:22.967102
17:46:23.219 INFO qallse.dsmaker.dsmaker: Doublets (len=1352) generated in f/var/folders/qp/grrqv98s1c37f8bxtkblfpcr0000gn/T/tmpra3o223v/ez-0.01_hpt-1.0/event000001000.


In [4]:
with open(path + '-meta.json') as f:
    print(f.read())

{
    "num_hits": 524,
    "num_tracks": 81,
    "num_important_tracks": 4,
    "num_noise": 130,
    "random_seed": 1547862382,
    "time": "2019-01-18T17:46:22.967102",
    "params": {
        "input_path": "/Users/lin/git/quantum-annealing-project/project-2/src/hepqpr/qallse/dsmaker/data/event000001000",
        "output_path": "/var/folders/qp/grrqv98s1c37f8bxtkblfpcr0000gn/T/tmpra3o223v",
        "density": 0.01,
        "high_pt_cut": 1.0,
        "double_hits_ok": false,
        "gen_doublets": true,
        "prefix": null,
        "random_seed": null,
        "phi_bounds": null,
        "min_hits_per_track": 5
    }
}


# Execute the model

## Load the data

In [6]:
# load data
dw = DataWrapper.from_path(path)
doublets = pd.read_csv(path + '-doublets.csv')
if add_missing:
    doublets = dw.add_missing_doublets(doublets)
else:
    p, r, ms = dw.compute_score(doublets)
    print(f'got {len(doublets)}.')
    print(f'  Input precision (%): {p*100:.4f}, recall (%): {r*100:.4f}, missing: {len(ms)}')

got 1352 doublets.
  Input precision (%): 3.2200, recall (%): 100.0000


## Build the model

In [7]:
%%time

# instantiate qallse
model = model_class(dw, **extra_config)
# build the qubo
model.build_model(doublets=doublets)
Q = model.to_qubo()

17:46:33.161 DEBUG qallse.qallse_d0: using config:
17:46:33.162 DEBUG qallse.qallse_d0:     beamspot_center: (0, 0, 0)
17:46:33.163 DEBUG qallse.qallse_d0:     beamspot_width: 27.5
17:46:33.163 DEBUG qallse.qallse_d0:     cheat: False
17:46:33.164 DEBUG qallse.qallse_d0:     d0_denom: 3.0
17:46:33.165 DEBUG qallse.qallse_d0:     d0_factor: 0.5
17:46:33.165 DEBUG qallse.qallse_d0:     max_layer_span: 2
17:46:33.166 DEBUG qallse.qallse_d0:     min_qplet_path: 2
17:46:33.166 DEBUG qallse.qallse_d0:     num_multiplier: -1
17:46:33.167 DEBUG qallse.qallse_d0:     qplet_max_dcurv: 0.0001
17:46:33.167 DEBUG qallse.qallse_d0:     qplet_max_strength: -0.2
17:46:33.168 DEBUG qallse.qallse_d0:     qubo_bias_weight: 0
17:46:33.168 DEBUG qallse.qallse_d0:     qubo_conflict_strength: 1
17:46:33.169 DEBUG qallse.qallse_d0:     rz_power: 1
17:46:33.169 DEBUG qallse.qallse_d0:     strength_bounds: None
17:46:33.169 DEBUG qallse.qallse_d0:     tplet_max_curv: 0.0008
17:46:33.170 DEBUG qallse.qallse_d0: 

CPU times: user 127 ms, sys: 10.6 ms, total: 138 ms
Wall time: 132 ms


## Sample the QUBO

Here, we use [_qbsolv_](https://github.com/dwavesystems/qbsolv) with default arguments. This means only classical algorithms.

In [8]:
%%time
# execute the qubo
response = model.sample_qubo(Q=Q)

17:46:38.267 INFO qallse.qallse_d0: QUBO of size 852 sampled in 0.76s (seed 2025012979).


CPU times: user 383 ms, sys: 4 ms, total: 387 ms
Wall time: 387 ms


## Parse the results

In [9]:
# get all output doublets
all_doublets = model.process_sample(next(response.samples()))
# recreate tracks and resolve remaining conflicts
final_tracks, final_doublets = TrackRecreaterD().process_results(all_doublets)

17:46:40.892 INFO qallse.track_recreater: Found 0 conflicting doublets


# Evaluate the results

## Print statistics and scores

In [10]:
# stats about the qbsolv run
en0 = dw.compute_energy(Q)
en = response.record.energy[0]

print(f'SAMPLE -- energy: {en:.4f}, ideal: {en0:.4f} (diff: {en-en0:.6f})')
occs = response.record.num_occurrences
print(f'          best sample occurrence: {occs[0]}/{occs.sum()}')

# scores
p, r, missings = dw.compute_score(final_doublets)
print(f'SCORE  -- precision (%): {p * 100}, recall (%): {r * 100}, missing: {len(missings)}')
trackml_score = dw.compute_trackml_score(final_tracks)
print(f'          tracks found: {len(final_tracks)}, trackml score (%): {trackml_score * 100}')

SAMPLE -- energy: -52.6204, ideal: -52.6204 (diff: 0.000000)
          best sample occurrence: 27/56
SCORE  -- precision (%): 100.0, recall (%): 100.0, missing: 0
          tracks found: 14, trackml score (%): 100.0


### Plot the results

You have the following at your disposal:
```python
iplot_results(dw, final_doublets, missings, dims=list('xy'))
iplot_results_tracks(dw, final_doublets, missings, dims=list('zr'))
iplot_results_any(dw, dw.real_doublets, dims=list('xy'))
```

For 3D plots, simply define `dims` as 3-dimensional. For example:
```python
iplot_results(dw, final_doublets, missings, dims=list('zxy'))
```

In [11]:
iplot_results(dw, final_doublets, missings, dims=list('xy'))

In [13]:
iplot_results(dw, final_doublets, missings, dims=list('zr'))

In [12]:
iplot_results_tracks(dw, final_tracks, dims=list('zxy'))